# 0. Configs and Imports

## 0.1. Imports

In [1]:
# Import for directory creation
from src.utils.file_operators import create_directory_structure

# Imports for data fetch
from src.data_collectors.yahoo_fin_collector import fetch_data

# Imports for loading config
from src.utils.file_operators import load_yaml

# Imports for Correlation Calculation
from src.computation.correlation import calculate_pairwise_corrrelation

# Imports for Portfolio Unierse Generation
from src.computation.ticker_universe_generator import create_portfolio_universe

## 0.2. Base file path, Configuration Filepath & Configuration Dictionary

In [2]:
base_file_path = "../data"

In [3]:
conf_path = "../config-dv.yaml"

In [4]:
config = load_yaml(conf_path)

## 0.3. Create Directory Structure (Optional: Run once only)

In [5]:
create_directory_structure(path=base_file_path, verbose=True)

INFO: Creating Base Directory
INFO: Base Directory path exists: ..\data
INFO: Creating Sub Directories
INFO: Sub Directory path exists: ..\data\01_raw

INFO: Sub Directory path exists: ..\data\02_processed

INFO: Sub Directory path exists: ..\data\03_analysis

SUCCESS: Created all directories successfully!



# 1. Data Fetch

## 1.1. Validate the configurations for data fecth

In [6]:
# Print the tickers defined in the config
ticker_list = config['config']['data']['tickers']

# Print the window and volitility_indicator defined in the config
window = config['config']['data']['window_in_days']
volatility_indicator = config['config']['data']['vix']

In [7]:
print("The tickers in the config list are:")

for idx, ticker in enumerate(ticker_list):
    print(f"    {idx+1}. {ticker}")

print(f"The window of data to evaluate is `{window}` days")

print(f"The volatility indicator used is `{volatility_indicator}`")

The tickers in the config list are:
    1. HDFCSML250.NS
    2. HDFCGOLD.NS
    3. HDFCNEXT50.NS
    4. AMBUJACEM.NS
    5. ASIANPAINT.NS
    6. EIDPARRY.NS
    7. ETERNAL.NS
    8. GMRAIRPORT.NS
    9. HDFCBANK.NS
    10. ABC.NS
    11. ITC.NS
    12. LLOYDSME.NS
    13. NBCC.NS
    14. OLECTRA.NS
    15. PCBL.NS
The window of data to evaluate is `900` days
The volatility indicator used is `^INDIAVIX`


## 1.2. Run the `fetch_data` pipeline

In [8]:
ohlcv_data = fetch_data(
    config_path=conf_path,
    save_data=True, 
    verbose=False
)

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ABC.NS"}}}
$ABC.NS: possibly delisted; no timezone found

1 Failed download:
['ABC.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: ABC.NS

SUCCESS: fetch_data pipeline run completed


# 2. Calculate Correlation

## 2.1. Validate the correaltion parameters

In [9]:
correlation_params = config['config']['parameters']['correlation']

In [10]:
print(f"The method used for calculating correlation is: {correlation_params['method']}")
print(f"Minimum samples needed for correlation are: {correlation_params['min_observations']}")
print(f"The price metrics defined inn the configs are:")
for idx, metric in enumerate(correlation_params['metrics']):
    print(f"    {idx+1}. {metric}")


The method used for calculating correlation is: spearman
Minimum samples needed for correlation are: 60
The price metrics defined inn the configs are:
    1. open
    2. high
    3. low
    4. close
    5. adj_close


## 2.2. Run the `calculate_pairwise_corrrelation` pipeline

In [11]:
correlation_results = calculate_pairwise_corrrelation(
    config_path=conf_path,
    save_data=True,
    verbose=True
)

INFO:    Starting Correlation Pipeline for open metric of data
INFO:    Calculating PCT Change for open metric of data
INFO:    Calculating Correlation for PCT Change for open metric of data
INFO:    Adjusting Correlation based on optimization strategy for open metric of data
SUCCESS:    Correlation Pipeline for open metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for open metric of data finished successfully

INFO:    Starting Correlation Pipeline for high metric of data
INFO:    Calculating PCT Change for high metric of data
INFO:    Calculating Correlation for PCT Change for high metric of data
INFO:    Adjusting Correlation based on optimization strategy for high metric of data
SUCCESS:    Correlation Pipeline for high metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for high metric of data finished successfully

INFO:    Starting Correlation Pipeline for low metric of data
INFO:    Calculating PCT Change for low metric of data
INFO:    Calculatin

## 2.3. Generate a portfolio Universe using the `create_portfolio_universe` function

In [52]:
# Standard Libs
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict

# Specialised imports
from src.utils.text_operators import format_error_text, format_info_text, format_success_text, format_warn_text
from src.utils.file_operators import load_latest_dataframe, load_yaml, save_dataframe
from src.utils.validators import validate_create_portfolio_universe_config, ConfigValidationError


def create_portfolio_universe(
        config_path: str | Path, 
        save_data: bool = True,
        verbose: bool = False
)  -> Dict[str, pd.DataFrame]:
    """
    Function: Imports the pair level correlation calculated for each ticker and creates a universe of all tickers with low correlation to select
    Args:
        config_path (str | Path): Path to the config file
        save_data (bool): Saves the data at specified path
        verbose (bool): Prints the operation status within the function
    """

    if verbose:
        print(format_info_text("Loading Portfolio Params"))

    # Load the config file
    config = load_yaml(Path(config_path))
    data_path_config = config['config']['data_path']
    corr_universe_params = config['config']['parameters']['portfolio_params']

    # Validate the config params
    try:
        validate_create_portfolio_universe_config(corr_universe_params)

    except ConfigValidationError as e:
        print(format_error_text(f"""Portfolio Parameters validation failed: {str(e)}"""))
        raise RuntimeError(format_error_text("Portfolio Universe Selection Pipeline Failed"))

    except Exception as e:
        print(format_error_text("Failed to load the correlation config parameters"))
        raise RuntimeError(format_error_text("Portfolio Universe Selection Pipeline Failed"))

    if verbose:
        print(format_success_text("Loaded and Validated Portfolio Params"))
        print()

    # Load the non filtered ticker x ticker matrix to calculate global corr correlation plot for adj_close
    corr_martix_df = load_latest_dataframe(
        directory=data_path_config['correlation_matrix']['directory'],
        file_name=data_path_config['correlation_matrix']['file_name'],
        file_format=data_path_config['correlation_matrix']['file_format'],
        suffix="adj_close"
    )

    # Process the correlation matrix
    corr_martix_df.rename(columns={'Unnamed: 0': 'securities'}, inplace=True)
    corr_martix_df.set_index(corr_martix_df['securities'], drop=True, inplace=True)
    corr_martix_df.drop(columns=['securities'], inplace=True)

    # Print the gloabl correlation caluclation has begun
    if verbose:
        print(format_success_text("Loaded the latest correlation Matrix"))
        print()
        print(format_info_text("Calculating the Global Correlation for all securities in correlation matrix"))

    # Calculate the absolute value for the correlation matrix, to eliminate directionality
    abs_corr_martix_df = np.abs(corr_martix_df)

    # Process values such that the diagonal elements are considered as NaN, to eliminate self correlation
    abs_corr_martix_df = abs_corr_martix_df.mask(
    np.eye(len(abs_corr_martix_df), dtype=bool)
)
    # Average Global correlation
    global_corr = abs_corr_martix_df.mean(axis=1)
    global_corr_sorted = global_corr.sort_values(ascending=True)
    anchors = global_corr_sorted
    
    # If the len of global_corr_sorted is less tham the anchor points specifed in the config raise a warning
    if len(global_corr_sorted) < corr_universe_params['correlation_anchor_points']:
        print(format_warn_text(f"The number of correlation_anchor_points is mmore than the legth of global correlation matrix, {len(global_corr_sorted)} anchor points will be used"))

    # Else filter the global correlation
    else:       
        if verbose:
            print(format_info_text(f"Filtering global correlation to loowest globally correlated {corr_universe_params['correlation_anchor_points']} anchor points"))
        anchors = anchors[:corr_universe_params['correlation_anchor_points']]

    # GC
    del abs_corr_martix_df
    del global_corr
    del corr_martix_df

    # Print the gloabl correlation caluclation has begun
    if verbose:
        print(format_success_text("Global Correlation for all securities caluclated"))
        print() 
        print(format_info_text("Finding Optimal Buckets"))
    
    return anchors, global_corr_sorted

In [56]:
anchors,global_corr_sorted = create_portfolio_universe(config_path=conf_path, save_data=True, verbose=True)

INFO: Loading Portfolio Params
SUCCESS: Loaded and Validated Portfolio Params

SUCCESS: Loaded the latest correlation Matrix

INFO: Calculating the Global Correlation for all securities in correlation matrix
INFO: Filtering global correlation to loowest globally correlated 5 anchor points
SUCCESS: Global Correlation for all securities caluclated

INFO: Finding Optimal Buckets


In [57]:
corr_matrix

(securities
 HDFCGOLD.NS      0.035472
 ITC.NS           0.188431
 ASIANPAINT.NS    0.190370
 ETERNAL.NS       0.236250
 HDFCBANK.NS      0.240647
 dtype: float64,
 securities
 HDFCGOLD.NS      0.035472
 ITC.NS           0.188431
 ASIANPAINT.NS    0.190370
 ETERNAL.NS       0.236250
 HDFCBANK.NS      0.240647
 EIDPARRY.NS      0.247220
 LLOYDSME.NS      0.271557
 PCBL.NS          0.296045
 OLECTRA.NS       0.305878
 GMRAIRPORT.NS    0.337130
 AMBUJACEM.NS     0.356156
 NBCC.NS          0.361313
 HDFCNEXT50.NS    0.406163
 HDFCSML250.NS    0.444957
 dtype: float64)

In [58]:
global_corr_sorted

securities
HDFCGOLD.NS      0.035472
ITC.NS           0.188431
ASIANPAINT.NS    0.190370
ETERNAL.NS       0.236250
HDFCBANK.NS      0.240647
EIDPARRY.NS      0.247220
LLOYDSME.NS      0.271557
PCBL.NS          0.296045
OLECTRA.NS       0.305878
GMRAIRPORT.NS    0.337130
AMBUJACEM.NS     0.356156
NBCC.NS          0.361313
HDFCNEXT50.NS    0.406163
HDFCSML250.NS    0.444957
dtype: float64